# http://tau-data.id/umi/

In [ ]:
# Installing Modules for Google Colab
!wget https://raw.githubusercontent.com/taufikedys/bps/master/taudata.py
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/slang.dic
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/stopwords_id.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/stopwords_en.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/stopwords_eng.txt
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/data.html
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/all_indo_man_tag_corpus_model.crf.tagger
!wget -P data/ https://raw.githubusercontent.com/taufikedys/umi/master/data/kata_dasar.txt
!pip install unidecode
!pip install pyLDAvis
!pip install textblob
!pip install sastrawi
!pip install twython
!pip install tweepy
!pip install spacy
!pip install python-crfsuite
!python -m spacy download en
!python -m spacy download xx
!python -m spacy download en_core_web_sm
import nltk
nltk.download('popular')

In [ ]:
# Importing Modules untuk Notebook ini
import warnings; warnings.simplefilter('ignore')
import taudata as tau, itertools, re, pickle, pyLDAvis, pyLDAvis.sklearn, spacy, nltk, urllib.request
import time, numpy as np, matplotlib.pyplot as plt, networkx as nx, pandas as pd, seaborn as sns 
from matplotlib.colors import ListedColormap
from tqdm import tqdm_notebook as tqdm
from textblob import TextBlob, Word
from nltk.tokenize import TweetTokenizer
from spacy.lang.id import Indonesian
from spacy.lang.en import English
from sklearn.datasets import fetch_20newsgroups
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from unidecode import unidecode
from nltk.tag import CRFTagger
from textblob import Word
from html import unescape
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from gensim.models import Word2Vec, FastText

factory = StopWordRemoverFactory()
ct = CRFTagger()
ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')
nlp_id = spacy.blank('id') #Indonesian()  # Language Model
nlp_en = spacy.load("en_core_web_sm") # English()
stemmer = StemmerFactory().create_stemmer()
pyLDAvis.enable_notebook()
sns.set(style="ticks", color_codes=True)
random_state = 170

<center><img alt="" src="images/0_Cover.jpg"/></center> 

## <center><font color="blue">Modul 09: Pendahuluan Social media Analytics</font></center>
<b><center>(C) Taufik Sutanto - 2019</center>
<center>tau-data Indonesia ~ https://tau-data.id ~ taufik@tau-data.id</center>

## <center><font color="blue">Pendahuluan Social Media Analysis</font></center>
<img alt="" src="images/PDS_logo.jpg" />

# <font color="blue">Workshop Schedule</font>

**Social Media & network Analysis**
* 09:00 – 11:00 	Topic Modelling
* 11:00 – 12:00	Latihan Topic Modelling
* 13:00 – 14:00	Pendahuluan Social Network Analytics
* 14:00 – 15.00	Centrality Analysis dan Community Detection
* 15:00 – 16.00	Latihan Centrality Analysis dan Community Detection 

Studi Kasus: **Sentiment Analysis dan Topic Modelling Kasus Banjir**


## <font color="blue">Outline Topic Modelling :</font>
* Pendahuluan Topic Modelling
* Soft Clustering (Topic Modelling): LDA dan NMF
* Visualisasi dan Interpretasi

<h3>Clustering?</h3>

<p><img alt="" src="images/5_what_is_clustering.png" style="height:457px; width:864px" /></p>


<p><img alt="" src="images/5_saveme_clustering.jpg" style="height:400px; width:400px" /></p>


<h3>Ketika mengolah dokumen (file dalam bentuk teks), harapan kita seperti ini:</h3>

<img alt="" src="images/4_harapan_LSA.png" style="height:99px; width:198px" />


<h3>Namun kita sudah bahas kemarin kenyataannya seperti ini:</h3>

<p><img alt="" src="images/4_kenyataan_LSA.png" style="height:183px; width:182px" /></p>

<h2 id="Topic-Modelling-1-:-Latent-Dirichlet-Allocation">Topic Modelling 1 : Latent Dirichlet Allocation</h2>

<p><img alt="" src="images/4_Document_to_Topics.png" style="height: 300px ; width: 582px" /></p>

<p><strong><big>Tapi bukan seperti klasifikasi dan bukan berarti kata-kata Sport, Technology, dan Entertainment dominan di kategori-kategori tersebut. Topic modelling lebih ke soft-clustering, dimana suatu dokumen dimasukkan ke dalam beberapa cluster (topic) sekaligus. Adapun nama &quot;topic/cluster&quot;-nya di interpretasi dari kata-kata yang ada didalamnya.</big></strong></p>


<p><img alt="" src="images/4_LDA vs LDA.JPG" style="height:400px; width:606px" /></p>
[<a href="http://chdoig.github.io/pytexas2015-topic-modeling/" target="_blank">Sumber gambar ini dan beberapa gambar selanjutnya</a>]</p>

<p><img alt="" src="images/4_definisi topic model.JPG" style="height:350px; width:809px" />


<p><img alt="" src="images/4_inti_LDA.JPG" style="height:500px; width:785px" /></p>
Penjelasan intuitif yang baik: https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d 

<h3>Evaluasi LDA?</h3>

<p><img alt="" src="images/4_Evaluasi_LDA.jpg" style="height:400px; width:888px" /></p>
[Cara lain: http://mimno.infosci.cornell.edu/slides/details.pdf]

<p><img alt="" src="images/4_LDA Pipeline.JPG" style="height:300px; width:663px" /></p>
* Modifikasi dapat dilakukan dengan "pos tags"

In [ ]:
# Kita mulai dengan membuat VSM-nya
# kita gunakan perintah yang ada di Segmen sebelumnya 
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer()

tf = tf_vectorizer.fit_transform(data)
tf_terms = tf_vectorizer.get_feature_names()
# Mengapa tf bukan tfidf?
# Blei, D. M., Ng, A. Y., & Jordan, M. I. (2003). Latent dirichlet allocation. Journal of machine Learning research, 3(Jan), 993-1022.
# Saran: untuk penelitian VS di Industri.
tf.shape

In [ ]:
# Dilanjutkan dengan membentuk model LDA-nya
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 3
lda = LDA(n_components=n_topics, learning_method='batch', random_state=0).fit(tf)   
lda

In [ ]:
# Melihat Topik-topiknya
vsm_topics = lda.transform(tf)
print(vsm_topics.shape)
vsm_topics[:10]
# Ukuran kolom = #Topics ==> Dimension Reduction

In [ ]:
# Seandainya diasumsikan 1 dokumen hanya 1 topic dengan nilai skor topic terbesar
doc_topic =  [a.argmax()+1 for a in tqdm(vsm_topics)] # topic of docs
doc_topic[:10]

In [ ]:
# mari kita plot
sns.countplot(doc_topic)
plt.show()

In [ ]:
# Mari kita coba maknai masing-masing topic ini
Top_Words=7
print('Printing top {0} Topics, with top {1} Words:'.format(n_topics, Top_Words))
tau.print_Topics(lda, tf_terms, n_topics, Top_Words)

In [ ]:
# %matplotlib inline
# Mari kita Plot, supaya lebih jelas
# Catatan, bergantung dari laptop yang digunakan, image terkadang cukup lama untuk muncul.
import pyLDAvis, pyLDAvis.sklearn; pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)  

# Bagaimana jika kita ingin menggunakan semi-supervised (guided) LDA?
https://medium.freecodecamp.org/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164 

<p>&nbsp;</p>

<h3 id="Non-Negative-Matrix-Decomposition-(NMF)">Non-Negative Matrix Decomposition (NMF)</h3>

<p><img alt="" src="images/4_NMF.jpg" style="height: 349px; width: 600px;" /> [image source: <a href="https://www.slideshare.net/SebastianRuder/dynamic-topic-modeling-via-nonnegative-matrix-factorization-dr-derek-greene]">https://www.slideshare.net/SebastianRuder/dynamic-topic-modeling-via-nonnegative-matrix-factorization-dr-derek-greene]</a></p>

<p>&nbsp;</p>


<h3 id="Tujuan-NMF:">Tujuan NMF:</h3>

<p><img alt="" src="images/4_NMF_Goal.JPG" style="height: 363px; width: 600px;" /></p>


In [ ]:
# Langsung Aplikasi-nya
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(data)
nmf_model = NMF(n_components = 3, random_state=0)
nmf = nmf_model.fit(tfidf)

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tau.print_Topics(nmf, tfidf_feature_names, n_topics, Top_Words)

In [ ]:
# Sama seperti LDA kita bisa melihat distribusi topic setiap dokumen
vsm_topics = nmf.transform(tfidf)
vsm_topics[:10]

In [ ]:
# Seandainya diasumsikan 1 dokumen hanya 1 topic dengan nilai skor topic terbesar
doc_topic =  [a.argmax()+1 for a in tqdm(vsm_topics)] # topic of docs
print('In total there are {0} major topics, distributed as follows'.format(len(set(doc_topic))))
sns.countplot(doc_topic)
plt.show()

In [ ]:
# Perkalian Matrix-nya (Aljabar Linear)
W = nmf_model.fit_transform(tfidf)
H = nmf_model.components_
terms = tfidf_vectorizer.get_feature_names()
print(W.shape, H.shape, len(terms))
#print(terms[:5])

<p><strong>Latihan :</strong></p>

<ol>
	<li>Load data News20 dengan sembarang 4&nbsp;kategori berbeda.&nbsp;</li>
	<li>Lakukan preprocessing (termasuk lemma) dan pos tag (ambil hanya noun saja)</li>
	<li>Bandingkan hasil topic dari LDA, dan NMF dari data tersebut.</li>
	<li>Apakah hasilnya sudah coherence penjelasannya sesuai dengan kategori news20 yang dipilih?</li>
	<li>Buat visualisasi pyLDAvis-nya dan analisa lebih lanjut.</li>
</ol>


# <center><font color="blue"> End of Module 09

<hr />
<p><img alt="" src="images/1_meme.jpg" /></p>